In [1]:

def load_spectrum_with_uncertainty(file_path):
    """
    Carga un archivo .mca ignorando encabezados y basura final.
    Lee solo datos entre <<DATA>> y <<END>> o hasta donde sean válidos.

    Returns:
    - channels: np.array de canales generados automáticamente
    - spectrum_with_unc: np.array de ufloat con cuentas e incertidumbre de Poisson
    """
  

    counts = []
    data_section = False

    with open(file_path, 'r', encoding='latin1') as f:
        for line in f:
            line = line.strip()

            if '<<DATA>>' in line:
                data_section = True
                continue
            elif '<<END>>' in line:
                break

            if data_section:
                try:
                    value = float(line)
                    counts.append(value)
                except ValueError:
                    continue  # ignorar líneas basura

    counts = np.array(counts)
    channels = np.arange(len(counts))
    #channels = data[:, 0].flatten()
    uncertainties = Poisson_Incertidumbre(counts)
    spectrum_with_unc = unp.uarray(counts, uncertainties)

    return channels, spectrum_with_unc



def peak_fit(raw_data_path, background_path, 
             peak_configs, width=3.7,
             save_path="Imagenes/lmFit", unidades="canal", ylim=None):
    """
    Carga datos crudos y de fondo, aplica propagación de incertidumbre y 
    muestra un gráfico del espectro corregido. Luego analiza los picos
    en los rangos especificados con el método indicado para cada pico.

    Parameters:
    - raw_data_path (str): Ruta al archivo de datos crudos
    - background_path (str): Ruta al archivo de fondo
    - peak_configs (list): Lista de diccionarios con la configuración de cada pico.
                          Cada diccionario debe tener las claves:
                          - "min_range": Canal mínimo
                          - "max_range": Canal máximo
                          - "method": Método a usar (1, 2, 3 o 4)
                          Ejemplo: [{"min_range": 100, "max_range": 150, "method": 1}, ...]
    - width (float): Factor para calcular el ancho del segundo ajuste (default: 3.0)
    - save_path (str): Ruta opcional para guardar el gráfico
    - unidades (str): Unidades para el eje x (default: "canal")
    - ylim (tuple): Límites para el eje y (opcional)

    Returns:
    - all_results (list): Lista con los resultados de todos los segundos ajustes
    """

    print(f"\n➡️ Cargando datos con incertidumbre de Poisson...")
    channels_raw, raw_with_unc = load_spectrum_with_uncertainty(raw_data_path)
    channels_back, background_with_unc = load_spectrum_with_uncertainty(background_path)
    print(type(channels_raw))
    print(type(raw_with_unc))
    if len(raw_with_unc) != len(background_with_unc):
        raise ValueError("⚠️ Raw data and background must be the same length.")

    corrected_data = raw_with_unc - background_with_unc
    corrected_counts = unp.nominal_values(corrected_data)
    graficar_espectros(channels_raw,
                   unp.nominal_values(raw_with_unc),
                   unp.nominal_values(background_with_unc),
                   corrected_counts,
                   color='tab:blue',
                   titulo=r'$^{137}$Cs',
                   save_path=save_path)


    # Número de picos basado en la longitud de peak_configs
    num_peaks = len(peak_configs)

    # Lista para almacenar los resultados
    all_results = []

    print(f"\n🧾 Procesando {num_peaks} picos...")

    for i in range(num_peaks):
        print(f"\n--- Pico #{i+1} ---")
    
        # Obtener configuración del pico actual
        config = peak_configs[i]
        min_range = config["min_range"]
        max_range = config["max_range"]
        method = config["method"]
        title = config["title"]
    
        print(f"Usando rango: Canal mínimo = {min_range}, Canal máximo = {max_range}")
        print(f"Método seleccionado: {method}")
    
        try:
            # Seleccionar modelo según método especificado para este pico
            if method == 1:
                modelo = gaussian_rect
       
            elif method == 2:
                modelo = gaussian_erfc
  
            elif method == 3:
                modelo = gaussian_rect_2
       
            elif method == 4:
                modelo = gaussian_erfc_2
         
            else:
                raise ValueError(f"Método no válido para el pico #{i+1}. Usa 1, 2, 3 o 4.")
        
            # Convertir a índices de canal
            idx_min = int(np.searchsorted(channels_raw, min_range))
            idx_max = int(np.searchsorted(channels_raw, max_range))
        
            # Datos para ajuste
            x = channels_raw
            y_with_unc = corrected_data
        
            # Primer ajuste
            results, params = lmfit_custom_fit(x, y_with_unc, modelo, method, idx_min, idx_max)
            
            # Preparar datos para gráfica del primer ajuste
            x_data = x[idx_min:idx_max]
            y_nom = y_with_unc[idx_min:idx_max]
            #params = results.params
            x_ajuste = np.linspace(min_range, max_range, 500)
            
            # Calcular valores del modelo para la curva de ajuste
            if method == 1:
                y_ajuste = modelo(x_ajuste, **{k: params[k].value for k in ['b0', 'b1', 'M', 'mean', 'stdev']})
                recta = params['b0'].value + params['b1'].value * (params['mean'] - x_ajuste)
            elif method == 2:
                y_ajuste = modelo(x_ajuste, **{k: params[k].value for k in ['b0', 'b1', 'M', 'mean', 'stdev']})
                recta = params['b0'].value + (params['b1'].value/2)*erfc((x_ajuste - params['mean'])/(np.sqrt(2)*params['stdev']))
            elif method == 3:
                y_ajuste = modelo(x_ajuste, **{k: params[k].value for k in ['b0', 'b1', 'M1', 'M2', 'mean1', 'mean2', 'stdev1', 'stdev2']})
                recta = params['b0'].value + params['b1'].value * (0.5*(params['mean1'] + params['mean2']) - x_ajuste)
            elif method == 4:
                y_ajuste = modelo(x_ajuste, **{k: params[k].value for k in ['b0', 'b1', 'M1', 'M2', 'mean1', 'mean2', 'stdev1', 'stdev2']})
                # Promedio de dos funciones erfc para doble pico
                recta = params['b0'].value + (params['b1'].value) * 0.5 * (
                    erfc((x_ajuste - params['mean1'])/(np.sqrt(2)*params['stdev1'])) + 
                    erfc((x_ajuste - params['mean2'])/(np.sqrt(2)*params['stdev2']))
                )
            
            # Título automático para el primer ajuste
 
            
            # Graficar y mostrar Markdown report para el primer ajuste
            md = plot_and_markdown_fit_report(
                result=results,
                x=x_data,
                y_with_unc=y_nom,
                x_ajuste=x_ajuste,
                y_ajuste=y_ajuste,
                recta_ajuste=recta,
                titulo=title,
                method=method,
                unidades=unidades,
                ylim=ylim,
                save_path=save_path,
                show_plot=True
            )
            display(md)
            
            # Realizar segundo ajuste con nuevos rangos calculados
            # Para métodos de doble pico, usamos el promedio de medias y desviaciones
            if method in [1, 2]:
                parameters = [params[k].value for k in ['b0', 'b1', 'M', 'mean', 'stdev']]
                mean = parameters[3]
                stdev = parameters[4]             
                x0_new = int(mean - width * stdev)
                x1_new = int(mean + width * stdev)
            else:  # Para métodos 3 y 4 (doble pico)
                mean1 = params['mean1'].value
                mean2 = params['mean2'].value
                stdev1 = params['stdev1'].value
                stdev2 = params['stdev2'].value
                # Usar el promedio o el valor más grande para determinar el rango
                mean = (mean1 + mean2) / 2
                stdev = max(stdev1, stdev2)          
                x0_new = int(mean1 - width * stdev1)
                x1_new = int(mean2 + width * stdev2)

            
            print(f"Realizando segundo ajuste con rango: Canal mínimo = {x0_new}, Canal máximo = {x1_new}")
            
            # Convertir a índices de canal para el segundo ajuste
            idx_min_new = int(np.searchsorted(channels_raw, x0_new))
            idx_max_new = int(np.searchsorted(channels_raw, x1_new))
            
            # Segundo ajuste
            results_new , params_new = lmfit_custom_fit(x, y_with_unc, modelo, method, idx_min_new, idx_max_new)
            
            # Preparar datos para gráfica del segundo ajuste
            x_data_new = x[idx_min_new:idx_max_new]
            y_nom_new = y_with_unc[idx_min_new:idx_max_new]
            #params_new = results_new.params
            x_ajuste_new = np.linspace(x0_new, x1_new, 500)
            
            # Calcular valores del modelo y línea base para el segundo ajuste
            if method == 1:
                y_ajuste_new = modelo(x_ajuste_new, **{k: params_new[k].value for k in ['b0', 'b1', 'M', 'mean', 'stdev']})
                recta_new = params_new['b0'].value + params_new['b1'].value * (params_new['mean'] - x_ajuste_new)
            elif method == 2:
                y_ajuste_new = modelo(x_ajuste_new, **{k: params_new[k].value for k in ['b0', 'b1', 'M', 'mean', 'stdev']})
                recta_new = params_new['b0'].value + (params_new['b1'].value/2)*erfc((x_ajuste_new - params_new['mean'])/(np.sqrt(2)*params_new['stdev']))
            elif method == 3:
                y_ajuste_new = modelo(x_ajuste_new, **{k: params_new[k].value for k in ['b0', 'b1', 'M1', 'M2', 'mean1', 'mean2', 'stdev1', 'stdev2']})
                recta_new = params_new['b0'].value + params_new['b1'].value * (0.5*(params_new['mean1'] + params_new['mean2']) - x_ajuste_new)
            elif method == 4:
                y_ajuste_new = modelo(x_ajuste_new, **{k: params_new[k].value for k in ['b0', 'b1', 'M1', 'M2', 'mean1', 'mean2', 'stdev1', 'stdev2']})
                recta_new = params_new['b0'].value + (params_new['b1'].value) * 0.5 * (
                    erfc((x_ajuste_new - params_new['mean1'])/(np.sqrt(2)*params_new['stdev1'])) + 
                    erfc((x_ajuste_new - params_new['mean2'])/(np.sqrt(2)*params_new['stdev2']))
                )
            
            # Título automático para el segundo ajuste
           # titulo_new = f"Pico_{i+1}_{model_name}_segundo_ajuste"
            
            # Graficar y mostrar Markdown report para el segundo ajuste
            md_new = plot_and_markdown_fit_report(
                result=results_new,
                x=x_data_new,
                y_with_unc=y_nom_new,
                x_ajuste=x_ajuste_new,
                y_ajuste=y_ajuste_new,
                recta_ajuste=recta_new,
                titulo=title,
                method=method,
                unidades=unidades,
                ylim=ylim,
                save_path=save_path,
                show_plot=True
             )
            display(md_new)
            
            # Guardar resultados del segundo ajuste
            all_results.append(results_new)
            
        except Exception as e:
            print(f"❌ Error en el pico #{i+1}: {str(e)}")
            continue
    
    # Retornar todos los resultados del segundo ajuste
    return all_results

99
